In [1]:
import os
import numpy as np
import tensorflow as tf
import librosa
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt


2024-08-14 16:50:53.409972: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-14 16:50:53.689692: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-14 16:50:53.928913: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-14 16:50:54.132693: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-14 16:50:54.193725: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-14 16:50:54.640153: I tensorflow/core/platform/cpu_feature_gu

In [2]:
AUDIO_PATH = 'CryCorpusFinal'
CRY_FOLDER = os.path.join(AUDIO_PATH, 'cry/augmented')
NOTCRY_FOLDER = os.path.join(AUDIO_PATH, 'notcry')
IMG_SIZE = (64, 64)
BATCH_SIZE = 8
EPOCHS = 100


: 

In [3]:
def load_audio_files(folder):
    files = []
    for filename in os.listdir(folder):
        if filename.endswith('.wav'):
            files.append(os.path.join(folder, filename))
    return files

def compute_spectrogram(y, sr, n_fft=2048, hop_length=512):
    D = librosa.stft(y, n_fft=n_fft, hop_length=hop_length)
    D_dB = librosa.amplitude_to_db(np.abs(D), ref=np.max)
    return D_dB

cry_files = load_audio_files(CRY_FOLDER)
notcry_files = load_audio_files(NOTCRY_FOLDER)

data = []
labels = []

for file_path in cry_files:
    y, sr = librosa.load(file_path, sr=None)
    spectrogram = compute_spectrogram(y, sr)
    data.append(spectrogram.flatten())  # Flatten the spectrogram to a 1D array
    labels.append(1)

for file_path in notcry_files:
    y, sr = librosa.load(file_path, sr=None)
    spectrogram = compute_spectrogram(y, sr)
    data.append(spectrogram.flatten())  # Flatten the spectrogram to a 1D array
    labels.append(0)

# Convert to numpy arrays
X_data = np.array(data, dtype=np.float32)
y_data = np.array(labels, dtype=np.float32)

X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.2, random_state=42)


In [ ]:
class LogisticRegression(tf.Module):
    def __init__(self):
        self.w = tf.Variable(tf.random.uniform([X_train.shape[1], 1]), name='weights')
        self.b = tf.Variable(tf.zeros([1]), name='bias')
    
    def __call__(self, x):
        return tf.sigmoid(tf.matmul(x, self.w) + self.b)

def loss_fn(model, x, y):
    y_ = model(x)
    return tf.reduce_mean(tf.losses.binary_crossentropy(y, y_))

def train(model, X, y, learning_rate):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, X, y)
    gradients = tape.gradient(loss, [model.w, model.b])
    optimizer = tf.optimizers.SGD(learning_rate)
    optimizer.apply_gradients(zip(gradients, [model.w, model.b]))
    return loss


In [ ]:
# Instantiate the model
model = LogisticRegression()

# Training loop
num_epochs = 100
learning_rate = 0.01
losses = []

for epoch in range(num_epochs):
    loss = train(model, X_train, y_train, learning_rate)
    losses.append(loss.numpy())
    if epoch % 10 == 0:
        print(f'Epoch {epoch}: Loss: {loss.numpy()}')


In [ ]:
plt.plot(range(num_epochs), losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss over Time')
plt.show()


In [ ]:
# Predict on the validation set
y_pred_val = model(X_val)
y_pred_val = tf.round(y_pred_val)  # Round predictions to 0 or 1

# Calculate accuracy and F1 score
acc = accuracy_score(y_val, y_pred_val.numpy())
f1 = f1_score(y_val, y_pred_val.numpy())

print(f'Validation Accuracy: {acc}')
print(f'Validation F1 Score: {f1}')


In [ ]:
def plot_decision_boundary(model, X, y):
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1),
                         np.arange(y_min, y_max, 0.1))
    grid = np.c_[xx.ravel(), yy.ravel()]
    preds = model(grid).numpy().reshape(xx.shape)
    plt.contourf(xx, yy, preds, alpha=0.8)
    plt.scatter(X[:, 0], X[:, 1], c=y, edgecolors='k')
    plt.show()

# Assuming you have reduced X_val to 2D
plot_decision_boundary(model, X_val, y_val)
